# NEPTUNE Step 3 Training Module
This notebook walks through using machine learning to generate informative priors and then applying Bayesian inference with MCMC. Each section below explains the purpose of the code so you can follow along and adapt it to your own data.


## Environment Setup
The code relies on `emcee`, `corner`, `plotly`, and `hdbscan`. If these packages are not installed in your environment, run the following cell to install them.


### Imports and Global Variables
This section imports the required Python libraries and defines convenient global constants used throughout the training module.


In [ ]:
import warnings
warnings.filterwarnings("ignore", message="invalid value encountered in scalar subtract")
from emcee.moves import StretchMove
from sklearn.preprocessing import MinMaxScaler
import hdbscan

import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import emcee
import corner
import os
from mpl_toolkits.mplot3d import Axes3D  # for 3D plotting
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from matplotlib.colors import LogNorm
from sklearn.cluster import DBSCAN  # for clustering the MCMC samples
import plotly.express as px

import numpy as np
import matplotlib.pyplot as plt
import corner
from scipy.stats import gaussian_kde

import numpy as np
import matplotlib.pyplot as plt
import corner
from scipy.stats import gaussian_kde

import numpy as np
import matplotlib.pyplot as plt
import corner
from scipy.stats import gaussian_kde
from matplotlib.colors import Normalize

import numpy as np
import matplotlib.pyplot as plt
import corner
from scipy.stats import gaussian_kde
from matplotlib.colors import Normalize, ListedColormap
import multiprocessing
from multiprocessing import Pool
multiprocessing.set_start_method('fork', force=True)
import multiprocessing as mp

# Create a "clipped" version of inferno using only the 0.98 → 1.0 segment
inferno_limited = ListedColormap(plt.cm.inferno(np.linspace(0.7, 1.0, 256)))
# Global placeholders to avoid pickling large models
_MCMC_BIN = None
_MCMC_PARAM = None
_MCMC_MODEL = None
_MCMC_SCALER = None
_MCMC_AMP = None
_MCMC_FREQ = None

from emcee.moves import StretchMove
import numpy as np



### Custom StretchMove and Clustering Utilities
Here we implement custom sampling moves and clustering helpers for the MCMC algorithm, allowing efficient exploration and grouping of candidate solutions.


In [ ]:
class ClippedStretchMove(StretchMove):
    def __init__(self, a, min_p, max_p):
        super().__init__(a=a)
        self.min_p = min_p
        self.max_p = max_p

    def propose(self, ensemble, state):
        # emcee now returns (new_state, accepted)
        new_state, accepted = super().propose(ensemble, state)

        # Clip the period (parameter 0) directly in the coords array:
        new_state.coords[..., 0] = np.clip(
            new_state.coords[..., 0],
            self.min_p,
            self.max_p
        )

        return new_state, accepted



def cluster_hdbscan(samples, min_cluster_size=30):
    """
    samples : (N,3) array, already scaled to [0,1] or z-score
    returns: (clusters, clusterer) where
      clusters: list of dict {label, indices, mean, std, n_samples}
      clusterer: the fitted HDBSCAN object (has .outlier_scores_, etc)
    """
    model = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,
                            cluster_selection_method='eom',
                            prediction_data=True)
    labels = model.fit_predict(samples)
    clusters = []
    for lab in np.unique(labels):
        if lab == -1:
            continue
        idx = np.flatnonzero(labels == lab)
        pts = samples[idx]
        clusters.append({
            "label":      int(lab),
            "indices":    idx,
            "mean":       pts.mean(axis=0),
            "std":        pts.std(axis=0),
            "n_samples":  len(idx)
        })
    return clusters, model

def init_mcmc_globals(bin_name, pf, modelA, scA, actual_amp, actual_freq):
    """
    Initialize globals so that the worker processes can access them without pickling.
    """
    global _MCMC_BIN, _MCMC_PARAM, _MCMC_MODEL, _MCMC_SCALER, _MCMC_AMP, _MCMC_FREQ
    _MCMC_BIN = bin_name
    _MCMC_PARAM = pf
    _MCMC_MODEL = modelA
    _MCMC_SCALER = scA
    _MCMC_AMP = actual_amp
    _MCMC_FREQ = actual_freq


def _log_prob_fast(theta):
    """
    A thin wrapper that only takes theta and uses global variables to evaluate log-prob.
    """
    return log_prob_3params_layer1(theta,
                                   _MCMC_BIN,
                                   _MCMC_PARAM,
                                   _MCMC_MODEL,
                                   _MCMC_SCALER,
                                   _MCMC_AMP,
                                   _MCMC_FREQ)


### Percentile Corner Plot
Utility functions for producing corner plots with percentile labels are defined so we can visualize posterior distributions at a glance.


In [ ]:
def plot_percentile_corner(chain_flat_post, log_probs, labels, bins=50):
    """
    Generates a corner plot where the color of the data points is based on fit percentiles.
    Uses only the 0.98 → 1.0 segment of the inferno colormap (bright top).
    The colorbar is displayed from 0.0 → 1.0 (where lower percentiles = better fits).
    
    Parameters:
    - chain_flat_post: (N, ndim) array of post burn-in MCMC samples.
    - log_probs: Log-probabilities of the samples (higher = better fit).
    - labels: List of parameter names.
    - bins: Number of bins for the corner plot.
    """
    ndim = len(labels)
    import numpy as np

    # Convert eccentricity from log-scale to linear for visualization (if needed)
    chain_for_plot = chain_flat_post.copy()
    chain_for_plot[:, 1] = chain_for_plot[:, 1]  # Example placeholder if you want log10(e) → e

    # Convert log-probabilities into percentiles [0, 1] (0 = best fit, 1 = worst fit)
    # Higher log_prob → better fit → lower percentile
    # np.argsort(-log_probs) ranks best log_prob as index 0
    ranks = np.argsort(np.argsort(-log_probs))
    percentiles = ranks / (len(ranks) - 1)
    # compute the histogram‐mode
    mass_vals = chain_for_plot[:, 2]  
    counts, edges = np.histogram(mass_vals, bins=100)

    i_mode = np.argmax(counts)
    mode_mass = 0.5 * (edges[i_mode] + edges[i_mode+1])
    

    # Create the corner plot without scatter first
    fig_percentile = corner.corner(
        chain_for_plot,
        labels=labels,
        #truths=[None, None, mode_mass],
        #truth_kwargs={'linewidth': 0},
        bins=bins,
        plot_contours=False,
        plot_density=False
    )

    # Now overplot scatter with percentile-based coloring
    axes = np.array(fig_percentile.axes).reshape((ndim, ndim))
    norm = Normalize(vmin=0, vmax=1)  # colorbar range 0 → 1
    for i in range(ndim):
        for j in range(i):
            ax = axes[i, j]
            scatter = ax.scatter(
                chain_for_plot[:, j],
                chain_for_plot[:, i],
                c=percentiles,
                cmap=inferno_limited,  # Use the clipped inferno
                norm=norm,
                s=4,
                alpha=0.8
            )

    # Move colorbar outside the plot
    fig_percentile.subplots_adjust(right=0.8)
    cbar_ax = fig_percentile.add_axes([0.85, 0.3, 0.02, 0.4])
    cb = fig_percentile.colorbar(scatter, cax=cbar_ax)
    cb.set_label("Fit Percentile (0 = Best)")

    plt.title("Percentile-Based Corner Plot (Using 0.98 → 1.0 of Inferno)")
    plt.show()


import numpy as np
import matplotlib.pyplot as plt
import corner
from scipy.stats import gaussian_kde
from matplotlib.colors import Normalize, ListedColormap

import numpy as np
import matplotlib.pyplot as plt
import corner
from scipy.stats import gaussian_kde
from matplotlib.colors import Normalize, ListedColormap

import numpy as np
import matplotlib.pyplot as plt
import corner
from scipy.stats import gaussian_kde
from matplotlib.colors import Normalize, ListedColormap



### Convergence Plot
This portion adds plotting tools to track convergence of the MCMC chains and confirm that the sampler has stabilized.


In [ ]:
def plot_convergence_corner(chain_flat_post, labels, bins=50):
    """
    Demonstration of 'darker = more convergence' with extra axis margin.
    We explicitly set the corner plot range to include 30% more space
    around the min/max of the data in each dimension.
    """

    # --- Build custom colormap for top 10% of inferno, reversed ---
    top_vals = np.linspace(0.7, 1.0, 256)
    inferno_slice = plt.cm.inferno(top_vals)
    inferno_top10_r = ListedColormap(inferno_slice[::-1])

    ndim = len(labels)
    chain_for_plot = chain_flat_post.copy()

    # Example: convert log10(e) → e
    chain_for_plot[:, 1] = 10 ** chain_for_plot[:, 1]

    # Avoid singular KDE issues
    variances = np.var(chain_for_plot, axis=0)
    low_variance_mask = variances < 1e-10
    chain_for_plot[:, low_variance_mask] += np.random.normal(
        0, 1e-6, size=chain_for_plot[:, low_variance_mask].shape
    )

    # --- KDE for densities ---
    kde = gaussian_kde(chain_for_plot.T, bw_method='scott')
    densities = kde(chain_for_plot.T)

    # Example: log-scale densities, then keep top 70%
    densities = np.log1p(densities)
    density_threshold = np.percentile(densities, 70)
    mask = densities > density_threshold
    chain_filtered = chain_for_plot[mask]
    densities_filtered = densities[mask]

    # =================================================
    #  A) Compute per-dimension ranges with +30% margin
    # =================================================
    data_min = chain_for_plot.min(axis=0)
    data_max = chain_for_plot.max(axis=0)

    ranges = []
    for i in range(ndim):
        span = data_max[i] - data_min[i]
        if span <= 0:
            # If dimension is nearly constant, just pick a small fixed range
            pad = 1e-3
        else:
            # 30% extra space on each side
            pad = 0
        low = data_min[i] - pad
        high = data_max[i] + pad
        ranges.append((low, high))

    # --- Heatmap-based corner plot (optional) ---

    # compute the histogram‐mode
    counts, edges = np.histogram(mass, bins=100)
    i_mode = np.argmax(counts)
    mode_mass = 0.5 * (edges[i_mode] + edges[i_mode+1])

    fig_heat = corner.corner(
        chain_for_plot,
        bins=50,
        range=full_range,
        plot_density=True,
        plot_datapoints=False,
        smooth=1,
        smooth1d=1,
        fill_contours=True,
        contour_kwargs={},
        color=None,
        show_titles=True,
        title_fmt=".3f"
    )
    plt.title("Heatmap-Based Corner Plot")
    plt.show()

    # --- Scatter-based corner plot ---
    fig_scatter = corner.corner(
        chain_filtered,
        labels=labels,
        #truths=[None, None, mode_mass],
        #truth_kwargs={'linewidths': 0},
        bins=bins,
        range=ranges,      # <-- pass same expanded ranges
        plot_contours=False,
        plot_density=False
    )

    # Normalize so min density -> colormap(0.0) (bright) and max density -> colormap(1.0) (dark)
    norm = Normalize(vmin=densities_filtered.min(), vmax=densities_filtered.max())

    axes = np.array(fig_scatter.axes).reshape((ndim, ndim))
    for i in range(ndim):
        for j in range(i):
            ax = axes[i, j]
            scatter = ax.scatter(
                chain_filtered[:, j],
                chain_filtered[:, i],
                c=densities_filtered,
                cmap=inferno_top10_r,
                norm=norm,
                s=4,
                alpha=0.8
            )

    # Move colorbar
    fig_scatter.subplots_adjust(right=0.8)
    cbar_ax = fig_scatter.add_axes([0.85, 0.3, 0.02, 0.4])
    cb = fig_scatter.colorbar(scatter, cax=cbar_ax)
    cb.set_label("Convergence Intensity")

    plt.title("Scatter-Based Corner Plot")
    plt.show()




###############################################################################
# DBSCAN cluster function (unchanged)
###############################################################################


### Sampling Diagnostics
After sampling, diagnostic functions report statistics like acceptance rates and autocorrelation lengths to evaluate performance.


In [ ]:
def cluster_mcmc_samples(chain_samples, eps=0.1, min_samples=30):
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(chain_samples)
    labels = clustering.labels_
    clusters = []
    for label in set(labels):
        if label == -1:  # ignore noise points
            continue
        indices = np.where(labels == label)[0]
        cluster_samples = chain_samples[indices]
        mean = np.mean(cluster_samples, axis=0)
        std = np.std(cluster_samples, axis=0)
        clusters.append({"mean": mean, "std": std, "n_samples": len(cluster_samples), "indices": indices})
    return clusters

###############################################################################
# Standard Gelman-Rubin convergence diagnostic for a set of chains.
###############################################################################
def gelman_rubin(chains):
    """
    Computes the Gelman-Rubin statistic for a set of chains.
    Input:
      chains: numpy array with shape (n_steps, n_walkers, n_dim)
    Returns:
      rhat: numpy array of length n_dim with the R̂ values.
    """
    n_steps, n_walkers, n_dim = chains.shape
    rhat = np.empty(n_dim)
    for d in range(n_dim):
        # Get chain for parameter d: shape (n_steps, n_walkers)
        chain_d = chains[:, :, d]
        chain_means = np.mean(chain_d, axis=0)
        overall_mean = np.mean(chain_means)
        B = n_steps * np.var(chain_means, ddof=1)
        W = np.mean(np.var(chain_d, axis=0, ddof=1))
        Var_hat = ((n_steps - 1) / n_steps) * W + (B / n_steps)
        rhat[d] = np.sqrt(Var_hat / W)
    return rhat

###############################################################################
# Gelman-Rubin diagnostic computed on samples in one cluster
###############################################################################
def gelman_rubin_cluster(chain_post, cluster_indices, n_walkers):
    """
    Compute Rhat for a given cluster. The cluster_indices are indices from the 
    flattened chain (from chain_post.reshape(-1, ndim)). This function re-groups 
    the samples by walker.

    Parameters:
      chain_post: numpy array of shape (n_steps, n_walkers, n_dim)
      cluster_indices: 1D array of flattened indices (from DBSCAN clustering)
      n_walkers: number of walkers (as in chain_post.shape[1])
    
    Returns:
      rhat: R̂ values for this cluster (numpy array of length n_dim),
            or None if insufficient samples/walkers.
    """
    n_steps, n_walkers, n_dim = chain_post.shape

    # Build a dictionary: for each walker, store a list of samples (in order of appearance).
    samples_by_walker = {w: [] for w in range(n_walkers)}
    
    # Because the flattened chain was obtained with row-major order,
    # each index i corresponds to: step = i // n_walkers, walker = i % n_walkers.
    for idx in cluster_indices:
        step = idx // n_walkers
        walker = idx % n_walkers
        samples_by_walker[walker].append(chain_post[step, walker, :])
    
    # Only keep walkers with at least 2 samples
    valid_walkers = {w: np.array(samples) for w, samples in samples_by_walker.items() if len(samples) >= 2}
    
    if len(valid_walkers) < 2:
        # Not enough chains in this cluster to compute Rhat
        return None

    # Truncate each walker's chain to the minimum number of samples found
    min_samples = min(len(samples) for samples in valid_walkers.values())
    # Build an array of shape (min_samples, n_valid_walkers, n_dim)
    chains_cluster = []
    for w in valid_walkers:
        chain_w = valid_walkers[w][:min_samples]
        chains_cluster.append(chain_w)
    chains_cluster = np.stack(chains_cluster, axis=1)  # shape: (min_samples, n_valid_walkers, n_dim)
    rhat = gelman_rubin(chains_cluster)
    return rhat

###############################################################################
# Plot predicted vs. actual
###############################################################################


### Model Building Functions
Functions in this section train random forest models on different data bins to provide informed starting points for the Bayesian inference step.


In [ ]:
def plot_predicted_vs_actual(y_true, y_pred, model_name, target_names):
    for i, target in enumerate(target_names):
        plt.figure()
        plt.scatter(y_true[:, i], y_pred[:, i], alpha=0.7)
        plt.plot([min(y_true[:, i]), max(y_true[:, i])],
                 [min(y_true[:, i]), max(y_true[:, i])], 'r--')
        plt.xlabel(f"Actual {target}")
        plt.ylabel(f"Predicted {target}")
        plt.title(f"{model_name} - Predicted vs. Actual: {target}")
        plt.show()

###############################################################################
# Get bin mask
###############################################################################
def get_bin_mask(df_all, period_bin, ecc_bin):
    if period_bin == "p1":
        period_mask = (df_all["ratio"] >= 1.9) & (df_all["ratio"] <= 1.99)
    elif period_bin == "p2":
        period_mask = (df_all["ratio"] >= 2.01) & (df_all["ratio"] <= 2.1)
    elif period_bin == "p3":
        period_mask = (df_all["ratio"] >= 1.4) & (df_all["ratio"] <= 1.49)
    elif period_bin == "p4":
        period_mask = (df_all["ratio"] >= 1.51) & (df_all["ratio"] <= 1.6)
    elif period_bin == "p5":
        period_mask = (df_all["ratio"] >= 2.2) & (df_all["ratio"] <= 2.5)
    else:
        period_mask = (df_all["ratio"] > 2.5) & (df_all["ratio"] <= 2.8)

    if ecc_bin == "e1":
        ecc_mask = (df_all["out_ecc"] <= -2.0)
    elif ecc_bin == "e2":
        ecc_mask = (df_all["out_ecc"] > -2.0) & (df_all["out_ecc"] <= -1.5228)
    else:
        ecc_mask = (df_all["out_ecc"] > -1.5228) & (df_all["out_ecc"] <= np.log10(0.08))
    return period_mask & ecc_mask

###############################################################################
# Build ModelB for a bin
###############################################################################
def build_modelB_for_bin(df_all, period_bin, ecc_bin):
    print(f"\n=== build_modelB_for_bin => period='{period_bin}', eccentricity='{ecc_bin}'")
    mask_bin = get_bin_mask(df_all, period_bin, ecc_bin)
    mask_amp = df_all["AmpP1"] #<= 100
    df = df_all[mask_bin & mask_amp].copy()
    if df.empty:
        print("No data for this bin.")
        return None, None, None
    featB = ["star_m", "inn_p", "inn_m", "inn_ecc", "inn_inc", "inn_omega", "AmpP1", "DomP1"]
    targB = ["out_m", "out_p"]
    df.dropna(subset=(featB + targB), inplace=True)
    #if period_bin in ("p3", "p4"):
    #    df = df[df["DomP1"] < 10].copy()
    if df.empty or len(df) < 5:
        print("Not enough data after dropping NA for this bin.")
        return None, None, None
    X_tr, X_val, y_tr, y_val = train_test_split(df[featB], df[targB],
                                                test_size=0.2, random_state=42)
    scB = StandardScaler()
    X_tr_sc = scB.fit_transform(X_tr)
    X_val_sc = scB.transform(X_val)
    rfB = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    rfB.fit(X_tr_sc, y_tr)
    y_pred_val = rfB.predict(X_val_sc)
    plot_predicted_vs_actual(y_val.values, y_pred_val,
                             f"Model B ({period_bin}, {ecc_bin})", targB)
    print(f"Number of data points in this bin after final filtering: {len(df)}")

    mae_vals = []
    rmse_vals = []
    mafe_vals = []
    r2_vals = []
    for i, col_ in enumerate(targB):
        mae_ = mean_absolute_error(y_val[col_], y_pred_val[:, i])
        r2_  = r2_score(y_val[col_], y_pred_val[:, i])
        rmse_ = np.sqrt(mean_squared_error(y_val[col_], y_pred_val[:, i]))
        mafe_ = np.mean(np.abs((y_val[col_] - y_pred_val[:, i]) / (y_val[col_] + 1e-8)))
        print(f"ModelB({period_bin}, {ecc_bin}) - {col_}: MAE={mae_:.4f}, RMSE={rmse_:.4f}, MAFE={mafe_:.4f}, R2={r2_:.4f}")
        mae_vals.append(mae_)
        rmse_vals.append(rmse_)
        mafe_vals.append(mafe_)
        r2_vals.append(r2_)
    error_metrics_B = {"MAE": np.mean(mae_vals),
                       "RMSE": np.mean(rmse_vals),
                       "MAFE": np.mean(mafe_vals),
                       "R2": np.mean(r2_vals)}
    return rfB, scB, error_metrics_B

###############################################################################
# Build ModelA for a bin
###############################################################################
def build_modelA_for_bin(df_all, period_bin, ecc_bin):
    print(f"\n=== build_modelA_for_bin => period='{period_bin}', eccentricity='{ecc_bin}' ===")
    featA = ["star_m", "inn_p", "inn_m", "inn_ecc", "inn_inc", "inn_omega",
             "out_p", "out_m", "out_ecc", "out_inc", "out_omega"]
    targA = ["AmpP1", "DomP1"]
    mask_bin = get_bin_mask(df_all, period_bin, ecc_bin)
    mask_amp = df_all["AmpP1"] #<= 100
    
    df = df_all[mask_bin & mask_amp].copy()
    

    #if period_bin in ("p3", "p4"):
    #    df = df[df["DomP1"] < 10].copy()
        
    if df.empty or len(df) < 5:
        print("Not enough data for this bin.")
        return None, None, None
    df.dropna(subset=(featA + targA), inplace=True)
    if df.empty:
        print("Not enough data after dropping NA for this bin.")
        return None, None, None
    X_tr, X_val, y_tr, y_val = train_test_split(df[featA], df[targA],
                                                test_size=0.2, random_state=42)
    scA = StandardScaler()
    X_tr_sc = scA.fit_transform(X_tr)

    # —— new: prevent divide-by-zero on constant features —— 
    # any scale_ that’s zero becomes 1.0
    zero_scale = scA.scale_ == 0
    if zero_scale.any():
        scA.scale_[zero_scale] = 1.0

    X_val_sc = scA.transform(X_val)
    rfA = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=42
    )

    rfA.fit(X_tr_sc, y_tr)

    y_pred_val = rfA.predict(X_val_sc)
    plot_predicted_vs_actual(y_val.values, y_pred_val,
                             f"Model A ({period_bin}, {ecc_bin})", targA)
    print(f"Number of data points in this bin after final filtering: {len(df)}")


    mae_vals = []
    rmse_vals = []
    mafe_vals = []
    r2_vals = []
    for i, col_ in enumerate(targA):
        mae_ = mean_absolute_error(y_val[col_], y_pred_val[:, i])
        r2_  = r2_score(y_val[col_], y_pred_val[:, i])
        rmse_ = np.sqrt(mean_squared_error(y_val[col_], y_pred_val[:, i]))
        mafe_ = np.mean(np.abs((y_val[col_] - y_pred_val[:, i]) / (y_val[col_] + 1e-8)))
        print(f"ModelA({period_bin}, {ecc_bin}) - {col_}: MAE={mae_:.4f}, RMSE={rmse_:.4f}, MAFE={mafe_:.4f}, R2={r2_:.4f}")
        mae_vals.append(mae_)
        rmse_vals.append(rmse_)
        mafe_vals.append(mafe_)
        r2_vals.append(r2_)
    error_metrics_A = {"MAE": np.mean(mae_vals),
                       "RMSE": np.mean(rmse_vals),
                       "MAFE": np.mean(mafe_vals),
                       "R2": np.mean(r2_vals)}
    return rfA, scA, error_metrics_A

###############################################################################
# LOG-PROB for MCMC
###############################################################################


### MCMC Sampling Routines
Here we collect routines that execute the MCMC sampler itself, drawing from the posterior using priors informed by the machine learning models.


In [ ]:
def log_prob_3params_layer1(theta, bin_name, pf, modelA, scA, actual_amp, actual_freq):
    newD = copy.deepcopy(pf)
    newD['out_p'], newD['out_ecc'], newD['out_m'] = theta

    # Period prior
    if newD['out_p'] < pf['period_prior_min'] or newD['out_p'] > pf['period_prior_max']:
        return -np.inf
    # Ratio in correct bin
    ratio = newD['out_p'] / newD['inn_p']
    if ratio < pf['period_bin_min'] or ratio > pf['period_bin_max']:
        return -np.inf
    # Eccentricity prior
    if newD['out_ecc'] > 0:
        return -np.inf
    if bin_name == 'e1' and newD['out_ecc'] >= -2:
        return -np.inf
    if bin_name == 'e2' and (newD['out_ecc'] < -2 or newD['out_ecc'] > -1.5228):
        return -np.inf
    if bin_name == 'e3' and (newD['out_ecc'] <= -1.5228 or newD['out_ecc'] > np.log10(0.08)):
        return -np.inf
    # Mass prior
    if newD['out_m'] < pf['mass_prior_min'] or newD['out_m'] > pf['mass_prior_max']:
        return -np.inf

    # Likelihood via Model A (SSE)
    feats = ["star_m","inn_p","inn_m","inn_ecc","inn_inc","inn_omega",
             "out_p","out_m","out_ecc","out_inc","out_omega"]
    dfA = pd.DataFrame([[newD[f] for f in feats]], columns=feats)
    try:
        X_scA = scA.transform(dfA)
        pred = modelA.predict(X_scA)[0]
    except:
        return -np.inf

    da = pred[0] - actual_amp
    dfv = pred[1] - actual_freq
    return -0.5 * (da*da + dfv*dfv)


###############################################################################
# MCMC function with per-cluster Gelman-Rubin convergence diagnostics and
# computation of an evidence estimate (lnZ) for Bayes factor comparisons.
###############################################################################
def _log_prob_wrapper(theta, bin_name, pf, modelA, scA, actual_amp, actual_freq):
    return log_prob_3params_layer1(
        theta, bin_name, pf, modelA, scA, actual_amp, actual_freq
    )

def do_mcmc_3param_layer1(period_bin, bin_name, old_param,
                          start_p, p_std, start_m, start_e,
                          modelA, scA,
                          actual_amp, actual_freq,
                          nsteps=400, nwalkers=30):
    # Set priors
    pf=copy.deepcopy(old_param)
    bins={'p1':(1.9,1.99),'p2':(2.01,2.1),'p3':(1.4,1.49),'p4':(1.51,1.6),'p5':(2.2,2.5)}
    pf['period_bin_min'],pf['period_bin_max']=bins.get(period_bin,(2.5,2.8))
    pf['period_prior_min'],pf['period_prior_max']=start_p-3*p_std,start_p+3*p_std
    pf['mass_prior_min'],pf['mass_prior_max']=max(start_m/3,0),min(3*start_m,200)
    # init walkers
    ndim=3; nc=int(np.ceil(nwalkers*0.7)); nu=nwalkers-nc
    lp, hp=old_param['inn_p']*pf['period_bin_min'],old_param['inn_p']*pf['period_bin_max']
    
    if bin_name=='e1': le,he=-9,-2.001
    elif bin_name=='e2': le,he=-2,-1.5228
    else: le,he=-1.5228,np.log10(0.08)
        # ─── after ────────────────────────────────────────────────
    # cluster walkers near the guess
    p_c = np.clip(start_p + 0.1*np.random.randn(nc), lp, hp)
    e_c = np.clip(start_e + 0.1*np.random.randn(nc), le, he)

    # include exactly two walkers at the extremes
    if nu >= 2:
        p_u = np.concatenate([[lp, hp],
                              np.random.uniform(lp, hp, nu-2)])
        e_u = np.concatenate([[le, he],
                              np.random.uniform(le, he, nu-2)])
    else:
        p_u = np.random.uniform(lp, hp, nu)
        e_u = np.random.uniform(le, he, nu)

    # mix and shuffle
    p0 = np.concatenate([p_c, p_u]); np.random.shuffle(p0)
    e0 = np.concatenate([e_c, e_u]); np.random.shuffle(e0)

    # leave mass initialization as before (or do same trick if you like)
    m_c = np.clip(start_m + 4*(np.random.rand(nc)-0.5),
                  pf['mass_prior_min'], pf['mass_prior_max'])
    m_u = np.random.uniform(pf['mass_prior_min'],
                            pf['mass_prior_max'], nu)
    m0  = np.concatenate([m_c, m_u]); np.random.shuffle(m0)

    pos0 = np.column_stack([p0, e0, m0])
    # ───────────────────────────────────────────────────────────

    # run MCMC
    init_mcmc_globals(bin_name,pf,modelA,scA,actual_amp,actual_freq)
    pool=multiprocessing.Pool(multiprocessing.cpu_count())
    pmin = start_p - 3 * p_std
    pmax = start_p + 3 * p_std
    #move = ClippedStretchMove(a=2, min_p=pmin, max_p=pmax)
    
   # sampler = emcee.EnsembleSampler(nwalkers, ndim, _log_prob_fast,
   #                                 moves=move,
   #                                 pool=pool)
    sampler = emcee.EnsembleSampler(nwalkers, ndim, _log_prob_fast, a=2, pool=pool)

    sampler.run_mcmc(pos0,nsteps,progress=True)
    
    from emcee.autocorr import integrated_time, AutocorrError

    # 1) set burn-in and thinning
    burn = nsteps // 4
    thin = 1
    
    # 2) grab the post-burn chain
    chain_array = sampler.get_chain(discard=burn, thin=thin)  
    #    shape = (n_steps_post, n_walkers, ndim)
    
    # 3) jitter zero-variance dims to avoid divide-by-zero
    variances = np.var(chain_array, axis=(0,1))
    for dim in np.where(variances < 1e-8)[0]:
        chain_array[:,:,dim] += np.random.normal(0, 1e-6,
                                                  size=chain_array[:,:,dim].shape)
    
    # 4) compute autocorr time with fallback
    try:
        tau = integrated_time(chain_array, quiet=True)
    except AutocorrError:
        tau = np.ones(chain_array.shape[2])  # fallback to τ=1
    
    print("Autocorrelation times (τ) for [out_p, log10(e), out_m]:", tau)
    
    # 5) effective sample size and convergence
    n_steps_post = chain_array.shape[0]
    n_walkers   = chain_array.shape[1]
    neff = n_steps_post * n_walkers / tau
    print("Approximate effective sample size:", neff)
    conv = np.all(n_steps_post >= 50 * tau)
    print("Converged (50τ per walker)?", conv)
        
    pool.close(); pool.join()
    # trace
    chain_all=sampler.get_chain()
    cp=chain_all.copy(); cp[:,:,1]=10**cp[:,:,1]
    fig,ax=plt.subplots(ndim,1,sharex=True,figsize=(6,6))
    labs=['out_p','out_e','out_m']
    for i in range(ndim): ax[i].plot(cp[:,:,i],alpha=0.5); ax[i].set_ylabel(labs[i])
    ax[-1].set_xlabel('Step'); plt.tight_layout(); plt.show()
    # post burn-in
    chp=sampler.get_chain(discard=nsteps//3,thin=2)
    flat=chp.reshape(-1,ndim); lin_flat=flat.copy(); lin_flat[:,1]=10**lin_flat[:,1]
    # clustering
    # ---- after: no normalization ----
    idx_sub = np.random.choice(len(lin_flat),
                                size=min(3000, len(lin_flat)),
                                replace=False)
    lin_sub = lin_flat[idx_sub]
    raw_clusters, hdb = cluster_hdbscan(lin_sub, min_cluster_size=500)


    # map only the *actual* HDBSCAN clusters back to full‐chain indices
    clusters = []
    for rc in raw_clusters:
        clusters.append({
            "mean":      rc["mean"],
            "std":       rc["std"],
            "indices":   idx_sub[rc["indices"]],   # safe: rc["indices"] < len(idx_sub)
            "n_samples": rc["n_samples"]
        })

    # if HDBSCAN found nothing, fall back to the entire chain
    if not clusters:
        clusters = [{
            "mean":      lin_flat.mean(0),
            "std":       lin_flat.std(0),
            "indices":   np.arange(len(lin_flat)),  # full array
            "n_samples": len(lin_flat)
        }]

    print("Cluster labels:", np.unique(hdb.labels_))
    print("Outlier scores:", hdb.outlier_scores_[:10])
    
    feats = [
    'star_m','inn_p','inn_m','inn_ecc','inn_inc','inn_omega',
    'out_p','out_m','out_ecc','out_inc','out_omega'
    ]

    solutions=[]

    for c in clusters:
        mean_lin=c['mean']; std_lin=c['std']; idx=c['indices']
        p_m, p_s = mean_lin[0],std_lin[0]
        m_m, m_s = mean_lin[2],std_lin[2]
        e_lin, e_s_lin = mean_lin[1],std_lin[1]
        e_m=np.log10(e_lin) if e_lin>1e-12 else -9; e_s=e_s_lin/(e_lin*np.log(10)) if e_lin>0 else 0
        # SSE and TTV preds
        ttv=[]
        # use the original 'flat' chain (which is already in log‐ecc) instead of lin_flat

        d = copy.deepcopy(pf)
        d["out_p"], d["out_ecc"], d["out_m"] = c["mean"]
        Xm = pd.DataFrame([[d[f] for f in feats]], columns=feats)
        ttv_mean = modelA.predict(scA.transform(Xm))[0]
        ttv = np.array([ttv_mean])

        ttv=np.array(ttv); ap_m,ap_s=ttv[:,0].mean(),ttv[:,0].std(); dp_m,dp_s=ttv[:,1].mean(),ttv[:,1].std()
        solutions.append({'p_mean':(p_m,p_s),'mass_mean':(m_m,m_s),'e_mean':(e_m,e_s),'ttvAmp':(ap_m,ap_s),'ttvDomP':(dp_m,dp_s)})
    # plots
    
   # Get post burn-in chain in 3D: shape (n_steps_post, n_walkers, ndim)
    chain_post = sampler.get_chain(discard=nsteps//3, thin=2)  # shape (n_steps_post, n_walkers, ndim)
    n_walkers = chain_post.shape[1]
    flat_full = chain_post.reshape(-1, ndim)                  # true full flattened samples
    chain_flat = flat_full

    # you already selected a subset idx_sub for HDBSCAN:
    #   lin_sub = flat_full[idx_sub]; clusters = cluster_hdbscan(...)
    # now map each cluster’s local indices back to flat_full
   

    print("\n=== DBSCAN clustering results ===")
    for iC, c_ in enumerate(clusters):
        print(f"Cluster {iC}: {c_['n_samples']} samples")
        # pass the full 3D chain_post into the diagnostic
        rhat_cluster = gelman_rubin_cluster(chain_post, c_["indices"], n_walkers)
        if rhat_cluster is not None:
            print(f"  Cluster {iC} Gelman-Rubin R̂: {rhat_cluster}")
        else:
            print(f"  Cluster {iC}: Not enough samples per walker to compute R̂ reliably.")
    if not clusters:
        print("No clusters => fallback to overall chain mean.")
        # use the full 3D post–burn-in chain for R̂
        #chain_post = sampler.get_chain(discard=nsteps//2, thin=2)  # (n_steps_post, n_walkers, ndim)
        rhat_all = gelman_rubin(chain_post)
        print(f"Overall chain Gelman-Rubin R̂: {rhat_all}")
        # now still summarize your flat array for the “single solution” fallback
        chain_flat = chain_post.reshape(-1, ndim).copy()
        mean_ = np.mean(chain_flat, axis=0)
        std_  = np.std(chain_flat, axis=0)
        clusters = [{"mean": mean_, "std": std_, "n_samples": len(chain_flat),
                     "indices": np.arange(len(chain_flat))}]
    
    solutions = []
    for c_ in clusters:
        p_mean = c_["mean"][0]
        p_std_ = c_["std"][0]
        m_mean = c_["mean"][2]
        m_std_ = c_["std"][2]
        mean_lin_e = c_["mean"][1]
        std_lin_e  = c_["std"][1]
        if mean_lin_e<=1e-12:
            mean_log_e = -9.0
            std_log_e  = 0.0
        else:
            mean_log_e = np.log10(mean_lin_e)
            std_log_e  = std_lin_e / (mean_lin_e*np.log(10)) if mean_lin_e>0 else 0.0

        # TTV predictions for each sample in cluster
        indices_ = c_["indices"]
        cluster_samples_log = chain_flat[indices_]
        ttv_preds = []
        for samp_ in cluster_samples_log:
            newD_ = copy.deepcopy(pf)
            newD_["out_p"]   = samp_[0]
            newD_["out_ecc"] = samp_[1]
            newD_["out_m"]   = samp_[2]
            featA = ["star_m","inn_p","inn_m","inn_ecc","inn_inc","inn_omega",
                     "out_p","out_m","out_ecc","out_inc","out_omega"]
            rowA = [[ newD_["star_m"], newD_["inn_p"], newD_["inn_m"], newD_["inn_ecc"],
                      newD_["inn_inc"], newD_["inn_omega"],
                      newD_["out_p"], newD_["out_m"], newD_["out_ecc"],
                      newD_["out_inc"], newD_["out_omega"] ]]
            X_dfA = pd.DataFrame(rowA, columns=featA)
            X_scA = scA.transform(X_dfA)
            pred_ = modelA.predict(X_scA)[0]
            ttv_preds.append(pred_)
        ttv_preds = np.array(ttv_preds)
        predAmp_mean = np.mean(ttv_preds[:,0])
        predAmp_std  = np.std(ttv_preds[:,0])
        predDomP_mean = np.mean(ttv_preds[:,1])
        predDomP_std = np.std(ttv_preds[:,1])

        newD_final = copy.deepcopy(pf)
        newD_final["out_p"]   = p_mean
        newD_final["out_ecc"] = mean_log_e
        newD_final["out_m"]   = m_mean
        newD_final["predAmp"] = predAmp_mean
        newD_final["predDomP"] = predDomP_mean

        # Compute SSE
        featA = ["star_m","inn_p","inn_m","inn_ecc","inn_inc","inn_omega",
                 "out_p","out_m","out_ecc","out_inc","out_omega"]
        rowA_ = [[ newD_final["star_m"], newD_final["inn_p"], newD_final["inn_m"], newD_final["inn_ecc"],
                   newD_final["inn_inc"], newD_final["inn_omega"],
                   newD_final["out_p"], newD_final["out_m"], newD_final["out_ecc"],
                   newD_final["out_inc"], newD_final["out_omega"] ]]
        X_dfA_ = pd.DataFrame(rowA_, columns=featA)
        X_scA_ = scA.transform(X_dfA_)
        pred__ = modelA.predict(X_scA_)[0]
        da_ = pred__[0] - actual_amp
        df_ = pred__[1] - actual_freq
        sse_ = da_**2 + df_**2

        solutions.append({"final_param_dict": newD_final,
                          "p_mean": (p_mean, p_std_),
                          "e_mean": (mean_log_e, std_log_e),
                          "mass_mean": (m_mean, m_std_),
                          "ttvAmp": (predAmp_mean, predAmp_std),
                          "ttvDomP": (predDomP_mean, predDomP_std),
                          "SSE": sse_})
    print("\nConvergence solutions for this bin combo:")
    for iSol, sol_ in enumerate(solutions):
        fd = sol_["final_param_dict"]
        p_val, p_err = sol_["p_mean"]
        m_val, m_err = sol_["mass_mean"]
        e_log, e_err = sol_["e_mean"]
        e_val = 10**(e_log)
        e_err_lin = 10**(e_log + e_err) - 10**(e_log) if e_val>1e-12 else 0.0
        ttv_amp, ttv_amp_err = sol_["ttvAmp"]
        ttv_domP, ttv_domP_err = sol_["ttvDomP"]
        sse_ = sol_["SSE"]
        print(f"  Solution {iSol}:")
        print(f"    Outer Period = {p_val:.6f} ± {p_err:.6f}")
        print(f"    Outer Mass   = {m_val:.6f} ± {m_err:.6f}")
        print(f"    Outer Ecc (linear) = {e_val:.6f} ± {e_err_lin:.6f}")
        print(f"    Predicted TTV Amp   = {ttv_amp:.6f} ± {ttv_amp_err:.6f}")
        print(f"    Predicted TTV DomP  = {ttv_domP:.6f} ± {ttv_domP_err:.6f}")
        print(f"    SSE = {sse_:.6f}\n")

    # --- NEW: Compute Bayesian evidence using a harmonic mean estimator ---
    log_prob_post = sampler.get_log_prob(discard=nsteps//3, thin=2, flat=True)
    log_prob_post = log_prob_post[np.isfinite(log_prob_post)]
    if len(log_prob_post) > 0:
        L_values = np.exp(log_prob_post)
        Z_est = 1.0 / np.mean(1.0 / L_values)
        lnZ = np.log(Z_est)
    else:
        lnZ = -np.inf
    print(f"Estimated log evidence (lnZ) = {lnZ:.6f}")
    # --- END NEW ---

    # Corner plots for visualization (unchanged)
    # Labels for corner plot
    labels_3 = ["out_p", "out_e", "out_m"]
    
    # Call the function to plot chain_flat_post and scatter-based corner plots
    #plot_convergence_corner(chain_flat, labels_3)

    

    p_min, p_max = np.min(chain_flat[:,0]), np.max(chain_flat[:,0])
    m_min, m_max = np.min(chain_flat[:,2]), np.max(chain_flat[:,2])
    pad_p = 0.1*(p_max-p_min)
    pad_m = 0.1*(m_max-m_min)
    full_range = [(p_min-pad_p, p_max+pad_p),
                  (np.min(10**(chain_flat[:,1])), np.max(10**(chain_flat[:,1]))),
                  (m_min-pad_m, m_max+pad_m)]
    chain_flat_for_corner = chain_flat.copy()
    chain_flat_for_corner[:,1] = 10**(chain_flat_for_corner[:,1])
    labels_3 = ["out_p","out_e","out_m"]
    n_plot = min(300, len(chain_flat_for_corner))
    idx    = np.random.choice(len(chain_flat_for_corner), size=n_plot, replace=False)
    small_all = chain_flat_for_corner[idx]
    mass_vals = small_all[:, 2]  
    counts, edges = np.histogram(mass_vals, bins=100)
    i_mode = np.argmax(counts)
    mode_mass = 0.5 * (edges[i_mode] + edges[i_mode+1])

    fig_corner_all = corner.corner(small_all,
                                   labels=labels_3,
                                   #truths=[None, None, mode_mass],
                                   #truth_kwargs={'linewidths': 0},
                                   bins=100,
                                   range=full_range,
                                   color='red',
                                   show_titles=True,
                                   title_fmt=".3f")
    plt.title("Corner Plot: All Samples")
    plt.show()

    fig_heat = corner.corner(chain_flat_for_corner,
                             labels=labels_3,
                             #truths=[None, None, mode_mass],
                             #truth_kwargs={'linewidths': 0},
                             bins=100,
                             range=full_range,
                             plot_density=True,
                             plot_datapoints=False,
                             smooth=1,
                             smooth1d=1,
                             fill_contours=True,
                             contour_kwargs={},
                             color=None,
                             show_titles=True,
                             title_fmt=".3f")
    #plt.title("Corner Plot")
    plt.show()

    chain_flat_post = sampler.get_chain(discard=nsteps//3, thin=2, flat=True).copy()
    idx = np.random.choice(len(chain_flat_post), size=300, replace=False)
    chain_flat_post = chain_flat_post[idx]
   

    n_plot = min(300, len(chain_flat_post))
    idx    = np.random.choice(len(chain_flat_post), size=n_plot, replace=False)
    chain_flat_post = chain_flat_post[idx]
    #log_probs_post  = log_probs_post[idx]
    
    labels_3 = ["out_p", "out_e", "out_m"]
    chain_flat_post[:,1] = 10**(chain_flat_post[:,1])


    #plot_percentile_corner(chain_flat_post, log_probs_post, labels_3)

    n_post = min(300, len(chain_flat_post))
    idx_post = np.random.choice(len(chain_flat_post), size=n_post, replace=False)
    small_post = chain_flat_post[idx_post]

    
    #fig_corner_post = corner.corner(small_post,
    #                                labels=labels_3,
    #                                bins=50,
    #                                range=full_range,
    #                                color='blue',
    #                                show_titles=True,
   #                                 title_fmt=".3f")
   # plt.title("Corner Plot: Post Burn-In")
   # plt.show()

  
    
    
    # Extract log-probabilities of post burn-in samples
    
    
    # Call function to generate percentile-based corner plot
    return solutions, lnZ
###############################################################################
# SINGLE-LAYER pipeline
###############################################################################


### Pipeline and Visualization Helpers
Helper utilities are provided to run the full pipeline and generate summary plots, streamlining repeated analyses.


In [ ]:
def run_singlelayer_pipeline_for_system(sysdict, modelB_map, modelA_map):
    baseParam = copy.deepcopy(sysdict)
    combos_e = {}
    period_bins = ["p4","p3","p1","p2","p5","p6"]
    ecc_bins = ["e1","e2","e3"]
    for pbin in period_bins:
        for ebin in ecc_bins:
            (rfB, scB, errB) = modelB_map.get((pbin, ebin),(None,None,None))
            if rfB is None:
                if pbin=="p1":
                    guess_p = baseParam["inn_p"]*1.95
                elif pbin=="p2":
                    guess_p = baseParam["inn_p"]*2.05
                elif pbin=="p3":
                    guess_p = baseParam["inn_p"]*1.45
                elif pbin=="p4":
                    guess_p = baseParam["inn_p"]*1.44
                elif pbin=="p5":
                    guess_p = baseParam["inn_p"]*2.35
                else:
                    guess_p = baseParam["inn_p"]*2.65
                guess_m = 10.0
            else:
                featB = ["star_m","inn_p","inn_m","inn_ecc","inn_inc","inn_omega","AmpP1","DomP1"]
                rowB = [[ baseParam["star_m"], baseParam["inn_p"], baseParam["inn_m"],
                          baseParam["inn_ecc"], baseParam["inn_inc"], baseParam["inn_omega"],
                          baseParam["AmpP1"], baseParam["DomP1"] ]]
                X_dfB = pd.DataFrame(rowB, columns=featB)
                X_scB = scB.transform(X_dfB)
                outB_ = rfB.predict(X_scB)[0]
                guess_m = outB_[0]
                guess_p = outB_[1]
            
            # Force guess_p to be within the allowed period bin limits
            if pbin=="p1":
                allowed_low = baseParam["inn_p"] * 1.9
                allowed_high = baseParam["inn_p"] * 1.99
            elif pbin=="p2":
                allowed_low = baseParam["inn_p"] * 2.01
                allowed_high = baseParam["inn_p"] * 2.1
            elif pbin=="p3":
                allowed_low = baseParam["inn_p"] * 1.56666666667
                allowed_high = baseParam["inn_p"] * 1.65666666667
            elif pbin=="p4":
                allowed_low = baseParam["inn_p"] * 1.45
                allowed_high = baseParam["inn_p"] * 1.55
            elif pbin=="p5":
                allowed_low = baseParam["inn_p"] * 2.2
                allowed_high = baseParam["inn_p"] * 2.5
            else:
                allowed_low = baseParam["inn_p"] * 2.5
                allowed_high = baseParam["inn_p"] * 2.8
            
            guess_p = np.clip(guess_p, allowed_low, allowed_high)

            p_std_for_mcmc = 0.05*guess_p
            if p_std_for_mcmc<1e-6:
                p_std_for_mcmc = max(0.5, 0.05*baseParam["inn_p"])
            if ebin=="e1":
                guess_e = -2.5
            elif ebin=="e2":
                guess_e = -1.8
            else:
                guess_e = -1.3
            (rfA, scA, errA) = modelA_map.get((pbin, ebin),(None,None,None))
            sol_list, lnZ = do_mcmc_3param_layer1(period_bin=pbin, bin_name=ebin, old_param=baseParam,
                start_p=guess_p, p_std=p_std_for_mcmc,
                start_m=guess_m, start_e=guess_e,
                modelA=rfA, scA=scA,
                actual_amp=baseParam["AmpP1"], actual_freq=baseParam["DomP1"],
                nsteps=20000, nwalkers=200
                )
            combos_e[(pbin,ebin)] = {"solutions": sol_list, "lnZ": lnZ, "errA": errA, "errB": errB}
    combos_final=[]
    for (pbin,ebin), results in combos_e.items():
        combos_final.append({"bins":(pbin,ebin), "solutions": results["solutions"], "lnZ": results["lnZ"],
                             "errA": results["errA"], "errB": results["errB"]})
    return combos_final

###############################################################################
# 3D / scatter plot helpers (unchanged)
###############################################################################
def set_3d_axes_and_reverse_mass(ax):
    x1,x2= ax.get_xlim3d()
    y1,y2= ax.get_ylim3d()
    z1,z2= ax.get_zlim3d()
    ax.set_xlim3d(x1,x2)
    ax.set_ylim3d(y2,y1)
    ax.set_zlim3d(z1,z2)

def reverse_mass_axis_plotly(fig):
    fig.update_layout(scene=dict(yaxis=dict(autorange="reversed")))

def three_3d_diff_plots_with_mass_reversed(X,Y,Z, title_extra=""):
    fig1= plt.figure()
    ax1= fig1.add_subplot(111, projection='3d')
    ax1.scatter(X,Y,Z, c='b', marker='o')
    ax1.set_xlabel("X")
    ax1.set_ylabel("Y (mass reversed)")
    ax1.set_zlabel("Z")
    ax1.set_title(f"3D Scatter {title_extra}")
    plt.draw()
    set_3d_axes_and_reverse_mass(ax1)
    plt.show()
    if len(X)>=3:
        fig2= plt.figure()
        ax2= fig2.add_subplot(111, projection='3d')
        ax2.plot_trisurf(X,Y,Z, edgecolor='gray', linewidth=0.2, alpha=0.5)
        ax2.set_xlabel("X")
        ax2.set_ylabel("Y (mass reversed)")
        ax2.set_zlabel("Z")
        ax2.set_title(f"3D Wire/TriSurf {title_extra}")
        plt.draw()
        set_3d_axes_and_reverse_mass(ax2)
        plt.show()
    fig3= px.scatter_3d(x=X, y=Y, z=Z, title=f"Interactive 3D {title_extra}")
    reverse_mass_axis_plotly(fig3)
    fig3.show()

def plot_3d_actual_pred_rev(p_act, m_act, e_act, p_pred, m_pred, e_pred, sys_idx):
    X= [p_act, p_pred]
    Y= [m_act, m_pred]
    Z= [e_act, e_pred]
    figA= plt.figure()
    axA= figA.add_subplot(111, projection='3d')
    axA.scatter([p_act],[m_act],[e_act], c='b', s=50, label='Actual')
    axA.scatter([p_pred],[m_pred],[e_pred], c='r', s=50, label='Predicted')
    axA.set_xlabel("Period")
    axA.set_ylabel("Mass (reversed)")
    axA.set_zlabel("Ecc")
    axA.set_title(f"System {sys_idx} => 3D scatter reversed mass")
    plt.draw()
    set_3d_axes_and_reverse_mass(axA)
    axA.legend()
    plt.show()

    figB= plt.figure()
    axB= figB.add_subplot(111, projection='3d')
    axB.plot(X,Y,Z, c='gray')
    axB.scatter([p_act],[m_act],[e_act], c='b', s=50, label='Actual')
    axB.scatter([p_pred],[m_pred],[e_pred], c='r', s=50, label='Predicted')
    axB.set_xlabel("Period")
    axB.set_ylabel("Mass (reversed)")
    axB.set_zlabel("Ecc")
    axB.set_title(f"System {sys_idx} => wire reversed mass")
    plt.draw()
    set_3d_axes_and_reverse_mass(axB)
    axB.legend()
    plt.show()

    df_plot= pd.DataFrame({"Period":X,"Mass":Y,"Ecc":Z,"Type":["Actual","Predicted"]})
    figC= px.scatter_3d(df_plot, x="Period", y="Mass", z="Ecc", color="Type",
                        symbol="Type", title=f"System {sys_idx} => interactive reversed mass")
    reverse_mass_axis_plotly(figC)
    figC.show()

def single_scatter(paramName, dataList):
    if not dataList:
        return
    x_ = [d[0] for d in dataList]
    y_ = [d[1] for d in dataList]
    e_ = [d[2] for d in dataList]
    plt.figure()
    plt.errorbar(x_, y_, yerr=e_, fmt='o', color='k', ecolor='red', capsize=3)
    mn = min(min(x_), min(y_))
    mx = max(max(x_), max(y_))
    plt.plot([mn,mx],[mn,mx],'r--')
    plt.xlabel(f"Actual {paramName}")
    plt.ylabel(f"Pred {paramName}")
    plt.title(f"{paramName} => so far: {len(dataList)} systems")
    plt.show()

###############################################################################
# MAIN
###############################################################################


### Main Execution Logic
This cell ties all components together, demonstrating how to run the workflow on a sample planetary system.


In [ ]:
def main():
    print("=== LOADING & PREPARING MAIN DATA ===")
    df_all = pd.read_csv(r"/home/ubuntu/MCMC/ttv_dataset_multiple_params.csv")

    rename_map = {
        "Stellar Mass (Msun)": "star_m",
        "Inner Period (days)": "inn_p",
        "Inner Mass (Mearth)": "inn_m",
        "Inner Eccentricity": "inn_ecc",
        "Inner Inclination": "inn_inc",
        "Inner Omega": "inn_omega",
        "Outer Mass (Mearth)": "out_m",
        "Outer Period (days)": "out_p",
        "Outer Eccentricity": "out_ecc",
        "Outer Inclination": "out_inc",
        "Outer Omega": "out_omega",
        "Amplitude of Dominant Period Test (P1)": "AmpP1",
        "Dominant Period Planet 1": "DomP1"
    }
    df_all.rename(columns=rename_map, inplace=True, errors='ignore')

    # Filter out large masses, clamp ecc, log-transform
    df_all = df_all[df_all["out_m"] <= 150].copy()
    df_all["inn_ecc"] = df_all["inn_ecc"].clip(lower=0, upper=1.0)
    df_all["out_ecc"] = df_all["out_ecc"].clip(lower=0, upper=1.0)
    df_all["inn_ecc"] = np.log10(df_all["inn_ecc"].replace(0, 1e-9))
    df_all["out_ecc"] = np.log10(df_all["out_ecc"].replace(0, 1e-9))

    df_all["ratio"] = df_all["out_p"] / df_all["inn_p"]

    def ratio_in_any_period_bin(r):
        return ((1.9<=r<=1.99) or (2.01<=r<=2.1) or
                (1.4<=r<=1.49) or (1.51<=r<=1.6) or
                (2.2<=r<=2.5) or (2.5<r<=2.8))
    def ecc_in_any_bin(e_log):
        e_lin=10**(e_log)
        return ((0.0<=e_lin<=0.01) or (0.01<e_lin<=0.03) or (0.03< e_lin<=1.0))
    mask_ratio = df_all["ratio"].apply(ratio_in_any_period_bin)
    mask_ecc   = df_all["out_ecc"].apply(ecc_in_any_bin)
    df_filtered= df_all[mask_ratio & mask_ecc].copy()

    essential_cols= ["star_m","inn_p","inn_m","inn_ecc","inn_inc","inn_omega",
                     "out_m","out_p","out_ecc","out_inc","out_omega","AmpP1","DomP1","ratio"]
    df_filtered.dropna(subset=essential_cols, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    if len(df_filtered)==0:
        print("No data remains after filtering => abort.")
        return

    df_all= df_filtered
    print(f"Data filtered: {len(df_all)} rows remain.\n")

    # Build the 18 models
    df_tmp = df_all.copy()
    period_bins = ["p4","p1","p2","p3","p5","p6"]
    ecc_bins    = ["e1","e2","e3"]
    print("\n=== Building ModelB for period and eccentricity bins ===")
    modelB_map={}
    modelB_errors = []
    for pb in period_bins:
        for eb in ecc_bins:
            rfB, scB, errB = build_modelB_for_bin(df_tmp, pb, eb)
            modelB_map[(pb, eb)] = (rfB, scB, errB)
            if errB is not None:
                modelB_errors.append(errB)
    print("\n=== Building ModelA for period and eccentricity bins ===")
    modelA_map={}
    modelA_errors = []
    for pb in period_bins:
        for eb in ecc_bins:
            rfA, scA, errA = build_modelA_for_bin(df_tmp, pb, eb)
            modelA_map[(pb, eb)] = (rfA, scA, errA)
            if errA is not None:
                modelA_errors.append(errA)

    # Print overall aggregated error metrics for ModelA and ModelB (all bins)
    if modelA_errors:
        overall_A = {"MAE": np.mean([d["MAE"] for d in modelA_errors]),
                     "RMSE": np.mean([d["RMSE"] for d in modelA_errors]),
                     "MAFE": np.mean([d["MAFE"] for d in modelA_errors]),
                     "R2": np.mean([d["R2"] for d in modelA_errors])}
        print("\nOverall aggregated errors for Model A (all bins):")
        print(overall_A)
    if modelB_errors:
        overall_B = {"MAE": np.mean([d["MAE"] for d in modelB_errors]),
                     "RMSE": np.mean([d["RMSE"] for d in modelB_errors]),
                     "MAFE": np.mean([d["MAFE"] for d in modelB_errors]),
                     "R2": np.mean([d["R2"] for d in modelB_errors])}
        print("\nOverall aggregated errors for Model B (all bins):")
        print(overall_B)

    # Now filter out bins with period p5 and p6 and compute again.
    modelA_errors_filtered = [d for (k,d) in zip(modelA_map.keys(), modelA_errors) if k[0] not in {"p5","p6"}]
    modelB_errors_filtered = [d for (k,d) in zip(modelB_map.keys(), modelB_errors) if k[0] not in {"p5","p6"}]
    # Note: Because the errors lists were appended in the loop (in order of looping), an alternative is to loop over keys.
    # Here we re-loop over the keys:
    modelA_errors_filtered = []
    for (pb, eb), (rfA, scA, errA) in modelA_map.items():
        if pb not in {"p5", "p6"} and errA is not None:
            modelA_errors_filtered.append(errA)
    modelB_errors_filtered = []
    for (pb, eb), (rfB, scB, errB) in modelB_map.items():
        if pb not in {"p5", "p6"} and errB is not None:
            modelB_errors_filtered.append(errB)

    if modelA_errors_filtered:
        overall_A_filtered = {"MAE": np.mean([d["MAE"] for d in modelA_errors_filtered]),
                              "RMSE": np.mean([d["RMSE"] for d in modelA_errors_filtered]),
                              "MAFE": np.mean([d["MAFE"] for d in modelA_errors_filtered]),
                              "R2": np.mean([d["R2"] for d in modelA_errors_filtered])}
        print("\nOverall aggregated errors for Model A (bins excluding p5 and p6):")
        print(overall_A_filtered)
    if modelB_errors_filtered:
        overall_B_filtered = {"MAE": np.mean([d["MAE"] for d in modelB_errors_filtered]),
                              "RMSE": np.mean([d["RMSE"] for d in modelB_errors_filtered]),
                              "MAFE": np.mean([d["MAFE"] for d in modelB_errors_filtered]),
                              "R2": np.mean([d["R2"] for d in modelB_errors_filtered])}
        print("\nOverall aggregated errors for Model B (bins excluding p5 and p6):")
        print(overall_B_filtered)

    # Define a single system dictionary (using user-provided values)
    user_system = {
        "star_m": 0.93,
        "inn_m": 3.49494,
        "inn_p": 14.91095177,
        "inn_ecc": np.log10(1e-9) if 0.0<=1e-12 else 0.0,
        "inn_inc": 89.98,
        "inn_omega": 0.0,
        "AmpP1": 38.20506441,
        "DomP1": 17.64122894,
        "out_m": 16.4,
        "out_p": 30,
        "out_ecc": np.log10(1e-9) if 0.0<=1e-12 else 0.0,
        "out_inc": 90,
        "out_omega": 0.0
    }

    #user_system = {
     #   "star_m": 0.97,
      #  "inn_m": 17.3,
       # "inn_p": 7.64159,
       # "inn_ecc": np.log10(1e-9) if 0.0<=1e-12 else 0.0,
       # "inn_inc": 87.68,
       # "inn_omega": 0.0,
       # "AmpP1": 8.165742477,
       # "DomP1": 34.84390735,
       # "out_m": 16.4,
       # "out_p": 14.85888,
       # "out_ecc": np.log10(1e-9) if 0.0<=1e-12 else 0.0,
       # "out_inc": 88.07,
       # "out_omega": 0.0
    #}
    #user_system["inn_ecc"] = -9.0
    #user_system["out_ecc"] = -9.0
    user_system["ratio"] = user_system["out_p"] / user_system["inn_p"]

    # Run the single-layer pipeline on the user system
    combos_18 = run_singlelayer_pipeline_for_system(user_system, modelB_map, modelA_map)

    print("\n=== RESULTS for the single user-provided system ===")
    bayes_results = {}
    for co in combos_18:
        pbin, ebin = co["bins"]
        sol_list = co["solutions"]
        lnZ = co["lnZ"]
        bayes_results[(pbin, ebin)] = lnZ
        print(f"Bin=({pbin},{ebin}), #solutions={len(sol_list)}, lnZ = {lnZ:.6f}")
        for iSol, sol_ in enumerate(sol_list):
            fd = sol_["final_param_dict"]
            p_val, p_err = sol_["p_mean"]
            m_val, m_err = sol_["mass_mean"]
            e_log, e_std = sol_["e_mean"]
            e_lin = 10**(e_log)
            e_err_lin = 10**(e_log + e_std)- 10**(e_log) if e_lin>1e-12 else 0.0
            a_val = fd["predAmp"]
            f_val = fd["predDomP"]
            sse_  = sol_["SSE"]
            print(f"  solution {iSol}: out_p={p_val:.6f} ± {p_err:.6f}, "
                  f"out_m={m_val:.6f} ± {m_err:.6f}, e_lin={e_lin:.6f} ± {e_err_lin:.6f}, "
                  f"PredAmp={a_val:.6f}, PredDomP={f_val:.6f}, SSE={sse_:.6f}")
    # --- NEW: Compute and print Bayes factor differences (ΔlnZ) ---
    if bayes_results:
        best_lnZ = max(bayes_results.values())
        print("\n=== Bayes Factor (ΔlnZ) Summary ===")
        for bin_key, lnZ in bayes_results.items():
            delta_lnZ = lnZ - best_lnZ
            print(f"Bin {bin_key}: lnZ = {lnZ:.6f}, ΔlnZ = {delta_lnZ:.6f}")
    # --- END NEW ---

    print("\nDone. You have the MCMC solutions for the single system.")
    print("No final difference plots are produced since we are only analyzing one system.")
    print("If you need additional plots, you can adapt the code above accordingly.")

    # --- NEW: Save final MCMC solution inputs and error bars to a list ---
    global final_solution_inputs
    final_solution_inputs = []
    for co in combos_18:
        for sol in co["solutions"]:
            sol_dict = {}
            # Outer period and error
            sol_dict["outer_period"] = sol["p_mean"][0]
            sol_dict["outer_period_err"] = sol["p_mean"][1]
            # Outer mass and error
            sol_dict["outer_mass"] = sol["mass_mean"][0]
            sol_dict["outer_mass_err"] = sol["mass_mean"][1]
            # Outer eccentricity (converted from log to linear) and error
            e_log = sol["e_mean"][0]
            e_std = sol["e_mean"][1]
            sol_dict["outer_ecc"] = 10**(e_log)
            sol_dict["outer_ecc_err"] = (10**(e_log + e_std) - 10**(e_log))
            # SSE
            sol_dict["SSE"] = sol["SSE"]
            # Predicted dominant period and amplitude with errors
            sol_dict["pred_dom_period"] = sol["ttvDomP"][0]
            sol_dict["pred_dom_period_err"] = sol["ttvDomP"][1]
            sol_dict["pred_dom_amp"] = sol["ttvAmp"][0]
            sol_dict["pred_dom_amp_err"] = sol["ttvAmp"][1]
            # True values from the system dictionary
            sol_dict["trueAmp"] = user_system["AmpP1"]
            sol_dict["trueDomP"] = user_system["DomP1"]
            # Include the rest of the parameters from final_param_dict
            sol_dict.update(sol["final_param_dict"])
            final_solution_inputs.append(sol_dict)
    print("\nFinal MCMC solution inputs and error bars:")
    for sol in final_solution_inputs:
        print(sol)



### Run as Script
The notebook can also be executed as a standalone script; this final section handles argument parsing and kicks off the analysis.


In [ ]:
if __name__=="__main__":
    main()


## Code Overview
The previous code cell defines helper functions for clustering, model training, and running MCMC.
Key functions include:
- `build_modelB_for_bin` and `build_modelA_for_bin` for training Random Forest regressors on different period and eccentricity bins.
- `do_mcmc_3param_layer1` which performs the MCMC sampling with custom priors.
- `run_singlelayer_pipeline_for_system` that orchestrates the entire workflow for one planetary system.


In [ ]:
pip install emcee corner plotly tqdm hdbscan

## Sorting the Solutions
After the MCMC run we sort the resulting parameter sets by sum of squared errors (SSE) to inspect the best fits first.


In [ ]:
# --- NEW: Sort final MCMC solution inputs by SSE (smallest to largest) ---
sorted_solution_inputs = sorted(final_solution_inputs, key=lambda x: x['SSE'])

print("\nSorted MCMC solutions by SSE (smallest to largest):")
for sol in sorted_solution_inputs:
    # Displaying the relevant information for each solution
    print(f"System with SSE = {sol['SSE']:.6f}")
    print(f"  Outer Period = {sol['outer_period']:.6f} ± {sol['outer_period_err']:.6f}")
    print(f"  Outer Mass = {sol['outer_mass']:.6f} ± {sol['outer_mass_err']:.6f}")
    print(f"  Outer Eccentricity (linear) = {sol['outer_ecc']:.6f} ± {sol['outer_ecc_err']:.6f}")
    print(f"  Predicted TTV Amp = {sol['pred_dom_amp']:.6f} ± {sol['pred_dom_amp_err']:.6f}")
    print(f"  Predicted TTV Dominant Period = {sol['pred_dom_period']:.6f} ± {sol['pred_dom_period_err']:.6f}")
    print(f"  True TTV Amp = {sol['trueAmp']:.6f}")
    print(f"  True TTV Dominant Period = {sol['trueDomP']:.6f}")
    print(f"  Outer Mass (True) = {sol['out_m']:.6f}")
    print(f"  Outer Period (True) = {sol['out_p']:.6f}")
    print(f"  Outer Eccentricity (True) = {sol['out_ecc']:.6f}")
    print(f"  Star Mass (True) = {sol['star_m']:.6f}")
    print(f"  Inner Period (True) = {sol['inn_p']:.6f}")
    print(f"  Inner Mass (True) = {sol['inn_m']:.6f}")
    print(f"  Inner Eccentricity (True) = {sol['inn_ecc']:.6f}")
    print(f"  Inner Inclination (True) = {sol['inn_inc']:.6f}")
    print(f"  Inner Omega (True) = {sol['inn_omega']:.6f}")
    print("---")

## Filtering by SSE
Solutions with SSE below the 40th percentile are kept for further inspection. This helps focus on high quality fits.


In [ ]:
# --- NEW: Filter solutions with SSE below the 40th percentile ---
import numpy as np

# Extract all SSE values from the final solutions list
sse_values = [sol['SSE'] for sol in final_solution_inputs]
# Compute the 40th percentile for SSE
percentile_40 = np.percentile(sse_values, 40)

# Create a duplicate of the final_solution_inputs list with only solutions having SSE below the 40th percentile
filtered_solution_inputs = [sol.copy() for sol in final_solution_inputs if sol['SSE'] < percentile_40]

print("\nMCMC solutions with SSE below 40th percentile (SSE < {:.6f}):".format(percentile_40))
for sol in filtered_solution_inputs:
    print(f"SSE = {sol['SSE']:.6f}")
    print(f"  Outer Period = {sol['outer_period']:.6f} ± {sol['outer_period_err']:.6f}")
    print(f"  Outer Mass = {sol['outer_mass']:.6f} ± {sol['outer_mass_err']:.6f}")
    print(f"  Outer Eccentricity (linear) = {sol['outer_ecc']:.6f} ± {sol['outer_ecc_err']:.6f}")
    print(f"  Predicted TTV Amp = {sol['pred_dom_amp']:.6f} ± {sol['pred_dom_amp_err']:.6f}")
    print(f"  Predicted TTV Dominant Period = {sol['pred_dom_period']:.6f} ± {sol['pred_dom_period_err']:.6f}")
    print(f"  True TTV Amp = {sol['trueAmp']:.6f}")
    print(f"  True TTV Dominant Period = {sol['trueDomP']:.6f}")
    print(f"  Outer Mass (True) = {sol['out_m']:.6f}")
    print(f"  Outer Period (True) = {sol['out_p']:.6f}")
    print(f"  Outer Eccentricity (True) = {sol['out_ecc']:.6f}")
    print(f"  Star Mass (True) = {sol['star_m']:.6f}")
    print(f"  Inner Period (True) = {sol['inn_p']:.6f}")
    print(f"  Inner Mass (True) = {sol['inn_m']:.6f}")
    print(f"  Inner Eccentricity (True) = {sol['inn_ecc']:.6f}")
    print(f"  Inner Inclination (True) = {sol['inn_inc']:.6f}")
    print(f"  Inner Omega (True) = {sol['inn_omega']:.6f}")
    print("---")


## Analysing TTV Periods
The final section demonstrates how to use Lomb--Scargle periodograms to analyse the transit timing variation curves and relate them back to the orbital periods inferred from MCMC.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.timeseries import LombScargle
from scipy.signal import find_peaks
import math

# ---------------------------
# Define the inner period (exact) in days
inner_period =8.30864937


# --- NEW: Use final MCMC solution inputs if available ---
try:
    filtered_solution_inputs
except NameError:
    # If final_solution_inputs is not defined, use default hard-coded values.
    outer_periods = [57.890833, 40.0, 50.0]         # Replace with your values
    outer_period_errors = [0.890964, 0.7, 1.0]        # Replace with your uncertainties
    print("Using hard-coded outer period values.")
else:
    # Extract the outer period and uncertainty from each solution
    outer_periods = [sol["outer_period"] for sol in filtered_solution_inputs]
    outer_period_errors = [sol["outer_period_err"] for sol in filtered_solution_inputs]
    print("Using outer period values from final_solution_inputs:")
    for op, err in zip(outer_periods, outer_period_errors):
        print(f"Outer Period = {op:.3f} ± {err:.3f} (in days)")

# Compute the synodic period and its propagated uncertainty for each outer period.
# Formula (in days):
#   T_syn = 1/((1/inner_period) - (1/outer_period))
#   dT_syn/dT_outer = (1/T_outer^2) / (1/inner_period - 1/outer_period)^2
# Then convert to epochs by dividing by inner_period.
synodic_periods = []
synodic_errors = []

for T_outer, sigma_outer in zip(outer_periods, outer_period_errors):
    # Compute the synodic period in days
    denominator = (1.0 / inner_period) - (1.0 / T_outer)
    if denominator == 0:
        raise ValueError("Denominator in synodic period calculation is zero.")
    T_syn = 1.0 / denominator

    # Propagate the error (inner_period assumed exact)
    sigma_syn = sigma_outer / (T_outer**2 * denominator**2)
    
    # Convert synodic period and its uncertainty from days to epochs
    T_syn_epochs = T_syn / inner_period
    sigma_syn_epochs = sigma_syn / inner_period
    
    synodic_periods.append(T_syn_epochs)
    synodic_errors.append(sigma_syn_epochs)

    print(f"Outer Period = {T_outer:.3f} ± {sigma_outer:.3f} days -> Synodic Period = {T_syn_epochs:.3f} ± {sigma_syn_epochs:.3f} epochs")

# ---------------------------
# Load the Kepler-46 TTV curve from a CSV file.
# The CSV must have columns: "Epoch" and "O-C (days)"
ttv_df = pd.read_csv("F:/Kepler_1710_b.csv")
if not {"Epoch", "O-C (days)"}.issubset(ttv_df.columns):
    raise ValueError("CSV file must contain columns 'Epoch' and 'O-C (days)'.")

# Extract the arrays from the CSV
all_epochs = ttv_df["Epoch"].values
all_oc = ttv_df["O-C (days)"].values

# ----- SUBTRACTION STEP (exactly as in your provided code, but only one iteration) -----
# Convert O–C from days to minutes
ttv_f = all_oc * 24.0 * 60.0
epochs_f = all_epochs  # Use all data; no error filtering

# Mean-center the TTV data (in minutes)
y_data = ttv_f - np.mean(ttv_f)

# Perform a single iteration of Lomb–Scargle to subtract the dominant periodic signal
ls_current = LombScargle(epochs_f, y_data)
frequency, power = ls_current.autopower(
    minimum_frequency=1.0/20.0,
    maximum_frequency=0.5,
    samples_per_peak=10
)
periods_ls = 1.0 / frequency  # Periods in epochs

# Limit to periods less than 40 epochs
valid = periods_ls < 100
if not np.any(valid):
    print("No valid periods found in subtraction step. Using original signal.")
    cleaned_signal = y_data
else:
    periods_valid = periods_ls[valid]
    power_valid = power[valid]
    idx_max = np.argmax(power_valid)
    dom_period = periods_valid[idx_max]
    dom_power = power_valid[idx_max]
    fap = ls_current.false_alarm_probability(dom_power)
    print("\n--- Subtraction Step ---")
    print(f"Dominant Period (epochs) = {dom_period:.3f}")
    print(f"Power                  = {dom_power:.4f}")
    print(f"FAP                    = {fap:.4g}")
    
    # Subtract the best-fit sinusoid for the dominant period
    model = ls_current.model(epochs_f, frequency=1.0/dom_period)
    cleaned_signal = y_data - model

# Use the full epochs and the cleaned signal (still in minutes) for further analysis.
time_for_analysis = epochs_f

# ---------------------------
# Compute Lomb–Scargle Periodogram using the cleaned signal.
# Frequency is in cycles per epoch, so periods are in epochs.
ls = LombScargle(time_for_analysis, cleaned_signal)
frequency, power = ls.autopower(minimum_frequency=0.01, maximum_frequency=1.0, samples_per_peak=20)
periods = 1.0 / frequency  # Periods in epochs

# Plot the overall periodogram (for reference)
plt.figure(figsize=(8, 5))
plt.plot(periods, power, color='black', label="Lomb–Scargle Power")
plt.xlabel("Period (epochs)")
plt.ylabel("Power")
plt.title("Lomb–Scargle Periodogram for Kepler-46 TTV (Cleaned)")
plt.legend()
plt.tight_layout()
plt.show()

# ---------------------------
# For each computed synodic period (in epochs), plot the periodogram with markers and search for peaks.
for idx, (syn, syn_err) in enumerate(zip(synodic_periods, synodic_errors), start=1):
    print(f"\n--- Synodic Period Set {idx}: {syn:.3f} ± {syn_err:.3f} epochs ---")
    
    # Plot 1: Full periodogram with synodic period marker and shaded error range
    plt.figure(figsize=(8, 5))
    plt.plot(periods, power, color='black', label="Lomb–Scargle Power")
    plt.axvline(x=syn, color='red', linestyle='--', label=f"Synodic Period ({syn:.2f} epochs)")
    plt.axvspan(syn - syn_err, syn + syn_err, color='red', alpha=0.3, label="Synodic Error Range")
    plt.xlabel("Period (epochs)")
    plt.ylabel("Power")
    plt.title(f"Full Periodogram with Synodic Marker (Set {idx})")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(8, 5))
    plt.plot(periods, power, color='black', label="Lomb–Scargle Power")
    plt.axvline(x=syn, color='red', linestyle='--', label=f"Synodic Period ({syn:.2f} epochs)")
    plt.axvspan(syn - syn_err, syn + syn_err, color='red', alpha=0.3, label="Synodic Error Range")
    plt.xlabel("Period (epochs)")
    plt.xlim(0.5 * syn, 1.5 * syn)
    plt.ylabel("Power")
    plt.title(f"Periodogram with Synodic Marker (Set {idx})")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # ---------------------------
    # Search for peaks in the error range [syn - syn_err, syn + syn_err]
    region_mask = (periods >= (syn - syn_err)) & (periods <= (syn + syn_err))
    indices = np.where(region_mask)[0]
    
    if len(indices) == 0:
        print("No periodogram points found in the synodic error region.")
        continue
    
    # Find peaks within the region (using scipy.signal.find_peaks)
    region_power = power[indices]
    peak_rel_indices, _ = find_peaks(region_power)
    
    if len(peak_rel_indices) == 0:
        # If no peaks detected, use the highest power point in the region.
        best_rel_idx = np.argmax(region_power)
        global_idx = indices[best_rel_idx]
        peak_period = periods[global_idx]
        peak_power = power[global_idx]
        fap = ls.false_alarm_probability(peak_power)
        print(f"No peaks detected in the region. Highest point: Period = {peak_period:.3f} epochs, Power = {peak_power:.4f}, FAP = {fap:.4g}")
    else:
        # For each detected peak, compute its period, power, and FAP.
        print("Detected peaks in the synodic error region:")
        for rel_idx in peak_rel_indices:
            global_idx = indices[rel_idx]
            peak_period = periods[global_idx]
            peak_power = power[global_idx]
            fap = ls.false_alarm_probability(peak_power)
            print(f"  Peak at {peak_period:.3f} epochs with Power = {peak_power:.4f} and FAP = {fap:.4g}")

# --- Compute the best peak (maximum power) in the error region for each synodic period ---
best_peak_power_list = []
best_fap_list = []

for syn, syn_err in zip(synodic_periods, synodic_errors):
    # Define the error region mask.
    region_mask = (periods >= (syn - syn_err)) & (periods <= (syn + syn_err))
    indices = np.where(region_mask)[0]
    if len(indices) == 0:
        best_peak_power = None
        best_fap = None
    else:
        region_power = power[indices]
        # Identify peaks within the region.
        peak_rel_indices, _ = find_peaks(region_power)
        if len(peak_rel_indices) == 0:
            # No peaks detected: choose the maximum power point in the region.
            best_idx = indices[np.argmax(region_power)]
        else:
            # Choose the peak with the maximum power.
            candidate_indices = indices[peak_rel_indices]
            best_idx = candidate_indices[np.argmax(power[candidate_indices])]
        best_peak_power = power[best_idx]
        best_fap = ls.false_alarm_probability(best_peak_power)
    best_peak_power_list.append(best_peak_power)
    best_fap_list.append(best_fap)

# --- First grouped figure: All synodic period sets with Power and FAP info in a text box ---

n_syn = len(synodic_periods)
n_cols = 3
n_rows = math.ceil(n_syn / n_cols)

fig, axs = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 4 * n_rows))
axs = axs.flatten()  # Flatten for simple iteration

for i, (syn, syn_err, best_power, best_fap) in enumerate(zip(synodic_periods, synodic_errors, best_peak_power_list, best_fap_list)):
    ax = axs[i]
    # Plot the full periodogram (zoomed in).
    ax.plot(periods, power, color='black', label="Lomb–Scargle Power")
    ax.axvline(x=syn, color='red', linestyle='--', label=f"Synodic = {syn:.2f}")
    ax.axvspan(syn - syn_err, syn + syn_err, color='red', alpha=0.3, label="Error Range")
    ax.set_xlabel("Period (epochs)")
    ax.set_ylabel("Power")
    ax.set_title(f"Set {i+1}: {syn:.2f} epochs")
    ax.set_xlim(0.5 * syn, 1.5 * syn)
    ax.legend(fontsize='small')
    
    # Create a text box with the best peak power and FAP.
    if best_power is not None:
        text_str = f"Power: {best_power:.4f}\nFAP: {best_fap:.4g}"
    else:
        text_str = "No peak found"
    ax.text(0.05, 0.95, text_str, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.8))

# Remove any unused subplots.
for j in range(i+1, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()

# --- Second grouped figure: Only synodic period sets with FAP below 0.5 ---
filtered_indices = [i for i, fap in enumerate(best_fap_list) if (fap is not None and fap < 0.5)]

if len(filtered_indices) > 0:
    n_filtered = len(filtered_indices)
    n_cols_f = 3
    n_rows_f = math.ceil(n_filtered / n_cols_f)
    
    fig2, axs2 = plt.subplots(n_rows_f, n_cols_f, figsize=(6 * n_cols_f, 4 * n_rows_f))
    axs2 = axs2.flatten()
    
    for idx, i in enumerate(filtered_indices):
        syn = synodic_periods[i]
        syn_err = synodic_errors[i]
        best_power = best_peak_power_list[i]
        best_fap = best_fap_list[i]
        ax = axs2[idx]
        ax.plot(periods, power, color='black', label="Lomb–Scargle Power")
        ax.axvline(x=syn, color='red', linestyle='--', label=f"Synodic = {syn:.2f}")
        ax.axvspan(syn - syn_err, syn + syn_err, color='red', alpha=0.3, label="Error Range")
        ax.set_xlabel("Period (epochs)")
        ax.set_ylabel("Power")
        ax.set_title(f"Set {i+1}: {syn:.2f} epochs")
        ax.set_xlim(0.5 * syn, 1.5 * syn)
        ax.legend(fontsize='small')
        if best_power is not None:
            text_str = f"Power: {best_power:.4f}\nFAP: {best_fap:.4g}"
        else:
            text_str = "No peak found"
        ax.text(0.05, 0.95, text_str, transform=ax.transAxes, fontsize=10,
                verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.8))
    
    # Remove any extra subplots.
    for j in range(idx+1, len(axs2)):
        fig2.delaxes(axs2[j])
    
    plt.tight_layout()
    plt.show()
else:
    print("No synodic period sets with FAP below 0.5.")
# --- Print summary parameters for all synodic period sets ---
print("\n--- Summary of Synodic Period Sets and Parameters ---")
for i, (T_outer, sigma_outer, syn, syn_err) in enumerate(zip(outer_periods, outer_period_errors, synodic_periods, synodic_errors), start=1):
    # If filtered_solution_inputs is defined, attempt to extract additional parameters
    if 'filtered_solution_inputs' in globals() and len(filtered_solution_inputs) >= i:
        sol = filtered_solution_inputs[i-1]
        outer_mass = sol.get("outer_mass", "N/A")
        outer_ecc  = sol.get("outer_ecc", "N/A")
        SSE = sol.get("SSE", "N/A")
        # You can add more parameters here if needed, e.g.:
        # paramX = sol.get("paramX", "N/A")
    else:
        outer_mass = "N/A"
        outer_ecc  = "N/A"
        SSE = "N/A"
    print(f"Set {i}: Outer Period = {T_outer:.3f} ± {sigma_outer:.3f} days, "
          f"Synodic Period = {syn:.3f} ± {syn_err:.3f} epochs, "
          f"Outer Mass = {outer_mass}, Outer Ecc = {outer_ecc}, SSE = {SSE}")


import numpy as np
from scipy.integrate import quad

# Constants for conversion
MEARTH_TO_MSUN = 3.003e-6  # Earth mass to solar mass conversion factor

# Laplace coefficient b_{1/2}^{(0)}(α)
def laplace_b(alpha):
    integrand = lambda psi: (1 - 2 * alpha * np.cos(psi) + alpha**2)**(-0.5)
    result, _ = quad(integrand, 0, 2 * np.pi, limit=1000)
    return result / np.pi

# Compute f(α)
def f_of_alpha(alpha, eps=1e-5):
    b_plus = laplace_b(alpha + eps)
    b_minus = laplace_b(alpha - eps)
    d_b_d_alpha = (b_plus - b_minus) / (2 * eps)
    b_val = laplace_b(alpha)
    return alpha * d_b_d_alpha + 2 * alpha * b_val

# Known Parameters
inner_period = final_solution_inputs     # P_transit in days
V_syn = 20.785 / (24*60)    # TTV amplitude in days (converted from minutes)

print("\nEstimated Outer Mass (perturber) from TTV amplitude for each solution:")

# Only predict outer mass for sets with FAP < 0.5.
for i, sol in enumerate(filtered_solution_inputs):
    if best_fap_list[i] is None or best_fap_list[i] >= 0.5:
        print(f"Skipping outer mass prediction for set {i+1} due to high FAP ({best_fap_list[i]:.4g}).")
        continue

    M_star = sol["star_m"]                # Stellar mass in solar masses
    m_inner = sol["inn_m"] * MEARTH_TO_MSUN  # Convert inner mass from Mearth to Msun
    P_pert = sol["outer_period"]          # Outer planet period in days

    # Compute α
    alpha = (inner_period / P_pert) ** (2.0 / 3.0)

    # Calculate f(α)
    f_val = f_of_alpha(alpha)

    # Correct mass calculation formula:
    m_pert_est = (V_syn * np.pi * (M_star + m_inner)) / (inner_period * alpha * f_val)

    # Output results clearly with units
    print(f"SSE = {sol['SSE']:.6f}, Outer Period = {P_pert:.3f} days, TTV Amp = {V_syn*1440:.3f} min")
    print(f"  α = {alpha:.4f}, f(α) = {f_val:.4e}, Estimated Outer Mass = {m_pert_est:.6f} M_sun")
    print(f"    (Equivalent to {m_pert_est/MEARTH_TO_MSUN:.2f} Earth masses)")
